In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler
from sklearn.model_selection import train_test_split
import cv2 as cv

!pip install split-folders matplotlib opencv-python spicy
import splitfolders

from pathlib import Path


2024-02-26 21:41:12.838882: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-26 21:41:12.838971: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-26 21:41:12.954802: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# Define the directory paths
dataset_Dir = '/kaggle/input/cottonweedid15/CottonWeedID15'
print(os.listdir(dataset_Dir))

num_classes = len(os.listdir(dataset_Dir))
print(num_classes)

redataset_Dir = '/kaggle/working/Weeddata_Split/'
ratio_list = [0.65, 0.2, 0.15]

splitfolders.ratio(dataset_Dir, output=redataset_Dir, seed=1337, ratio=ratio_list, group_prefix=None, move=False)


['Nutsedge', 'Sicklepod', 'Morningglory', 'Ragweed', 'PalmerAmaranth', 'Waterhemp', 'Crabgrass', 'Swinecress', 'Prickly Sida', 'Carpetweeds', 'SpottedSpurge', 'SpurredAnoda', 'Eclipta', 'Goosegrass', 'Purslane']
15


Copying files: 5187 files [02:35, 33.38 files/s] 


In [3]:
# Define constants
IMAGE_SIZE = (512, 512)
COLOR_MODE = 'rgb'
BATCH_SIZE = 8  # Reduced batch size
NUM_CLASSES = 15

# Function to create data generators
def create_data_generators(redataset_Dir):
    train_dataset = tf.keras.utils.image_dataset_from_directory(
        os.path.join(redataset_Dir, 'train'),
        label_mode='categorical',
        image_size=IMAGE_SIZE,
        color_mode=COLOR_MODE,
        batch_size=BATCH_SIZE,
        shuffle=True,
        seed=42,
    )

    valid_dataset = tf.keras.utils.image_dataset_from_directory(
        os.path.join(redataset_Dir, 'val'),
        label_mode='categorical',
        image_size=IMAGE_SIZE,
        color_mode=COLOR_MODE,
        batch_size=BATCH_SIZE,
        shuffle=False,
    )

    test_dataset = tf.keras.utils.image_dataset_from_directory(
        os.path.join(redataset_Dir, 'test'),
        label_mode='categorical',
        image_size=IMAGE_SIZE,
        color_mode=COLOR_MODE,
        batch_size=BATCH_SIZE,
        shuffle=False,
    )

    return train_dataset, valid_dataset, test_dataset

# Create data generators
train_dataset, valid_dataset, test_dataset = create_data_generators(redataset_Dir)


Found 3364 files belonging to 15 classes.
Found 1031 files belonging to 15 classes.
Found 792 files belonging to 15 classes.


In [4]:
# Preprocess and prepare datasets
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
valid_dataset = valid_dataset.cache().prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.cache().prefetch(buffer_size=AUTOTUNE)

rescale_Func = tf.keras.layers.Rescaling(1./127.5, offset=-1)

normalized_train_dataset = train_dataset.map(lambda x, y: (rescale_Func(x), y))
normalized_valid_dataset = valid_dataset.map(lambda x, y: (rescale_Func(x), y))
normalized_test_dataset = test_dataset.map(lambda x, y: (rescale_Func(x), y))

data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(180)
])

normalized_train_dataset = normalized_train_dataset.map(lambda x, y: (data_augmentation(x), y))


In [5]:
# Build the model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(*IMAGE_SIZE, 3))

for layer in base_model.layers:
    layer.trainable = True

x = Flatten()(base_model.output)
x = Dense(1024, activation='relu')(x)  # Reduced units
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
sgd = SGD(learning_rate=0.001, momentum=0.9)
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])


9406464/9406464 [==============================] - 0s 0us/step


In [6]:
# Train the model
def lr_scheduler(epoch):
    initial_lr = 0.001
    decay_factor = 0.1
    epochs_drop = 7
    lr = initial_lr * np.power(decay_factor, np.floor((1 + epoch) / epochs_drop))
    return lr

lr_scheduler_callback = LearningRateScheduler(lr_scheduler)

history = model.fit(
    normalized_train_dataset,
    epochs=50,
    validation_data=normalized_valid_dataset,
    callbacks=[lr_scheduler_callback],
)


Epoch 1/50


I0000 00:00:1708983948.144471     117 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


421/421 [==============================] - 229s 313ms/step - loss: 1.6116 - accuracy: 0.5699 - val_loss: 2.6562 - val_accuracy: 0.3307 - lr: 0.0010
Epoch 2/50
421/421 [==============================] - 109s 258ms/step - loss: 0.6270 - accuracy: 0.8023 - val_loss: 3.8254 - val_accuracy: 0.3957 - lr: 0.0010
Epoch 3/50
421/421 [==============================] - 108s 256ms/step - loss: 0.4446 - accuracy: 0.8612 - val_loss: 1.0004 - val_accuracy: 0.7527 - lr: 0.0010
Epoch 4/50
421/421 [==============================] - 109s 258ms/step - loss: 0.3183 - accuracy: 0.8986 - val_loss: 0.6866 - val_accuracy: 0.8225 - lr: 0.0010
Epoch 5/50
421/421 [==============================] - 111s 264ms/step - loss: 0.2475 - accuracy: 0.9218 - val_loss: 1.6318 - val_accuracy: 0.7391 - lr: 0.0010
Epoch 6/50
421/421 [==============================] - 119s 282ms/step - loss: 0.2556 - accuracy: 0.9197 - val_loss: 0.7406 - val_accuracy: 0.8021 - lr: 0.0010
Epoch 7/50
421/421 [==============================] - 117

In [7]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(normalized_test_dataset)

print("Training Accuracy:", history.history['accuracy'][-1])
print("Validation Accuracy:", history.history['val_accuracy'][-1])
print("Test Accuracy:", test_accuracy)

# Save the model
model.save('MobileNetV2_custom_model.keras')


99/99 [==============================] - 20s 196ms/step - loss: 0.1809 - accuracy: 0.9545
Training Accuracy: 0.9922711253166199
Validation Accuracy: 0.9563530683517456
Test Accuracy: 0.9545454382896423
